# 朴素贝叶斯
***优点:*** 在数据较少的情况下仍然有效， 可以处理多分类问题  
***缺点:*** 对于输入数据的准备方式较为敏感  
***适用数据类型:*** 标称型数据

### 贝叶斯决策理论
$p(c_i|x,y) = \frac{p(x,y|c_i)p(c_i)}{p(x,y)}$  
贝叶斯分类准则：
* 如果$p(c_1|x,y) > p(c_2|x,y)$, 那么属于类别$c_1$
* 如果$p(c_1|x,y) < p(c_2|x,y)$, 那么属于类别$c_2$

### 朴素贝叶斯进行文档分类

测试数据

In [1]:
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 代表侮辱性文字, 0 代表正常言论
    return postingList,classVec

###### 从文本中构建词向量

In [2]:
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

词集模型(set-of-words model)

In [3]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("The word: %s is not in my Vocabulary!" % word)
    return returnVec

测试

In [4]:
listOPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
len(myVocabList)

32

In [5]:
setOfWords2Vec(myVocabList, listOPosts[3])

[1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0]

###### 训练算法： 从词向量计算概率
$p(c_i|\mathbf{w})=\frac{p(\mathbf{w}|c_i)|p(c_i)}{p(\mathbf{w})}$  
假设所有词相互独立
$p(\mathbf{w})=p(w_0,w_1..w_N|c_i)=p(w_0|c_i)p(w_1|c_i)..p(w_N|c_i)$

伪代码:
```
计算每个类别中的文档数目
对每篇训练文档:
   对每个类别:
      如果词条出现在文档中->增加该词条的计数值
      增加所有词条的计数值
   对每个类别：
      对每个词条:
         将该词条的数目/总词条数目得到条件概率
   返回每个类别的条件概率
```

In [6]:
import numpy as np

In [7]:
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = np.sum(trainCategory)/float(numTrainDocs)
    p0Num = np.ones(numWords); p1Num = np.ones(numWords) # 防止出现0
    p0Denom = 2.0; p1Denom = 2.0 # 防止出现0， 乘积变0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num/p1Denom) # log防下溢
    p0Vect = np.log(p0Num/p0Denom)
    return p0Vect,p1Vect, pAbusive

In [8]:
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))

In [9]:
p0V, p1V, pAb = trainNB0(trainMat, listClasses)
pAb

0.5

###### 测试算法

In [10]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = np.sum(vec2Classify * p1Vec) + np.log(pClass1)
    p0 = np.sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

测试

In [11]:
testEntry = ['love', 'my', 'dalmation']
thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))
testEntry = ['stupid', 'garbage']
thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


词袋模型(bag-of-words model)

In [12]:
def bagOfWordsVecMN(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec